In [16]:
import numpy as np
import pickle
from scipy.io import savemat
import sys
sys.path.append('..')


In [17]:
expName = 'sweep'
chnNames = ['PZ', 'PO5', 'POZ', 'PO3', 'PO4',
            'PO6', 'PO7', 'PO8', 'O1', 'OZ', 'O2']

dir = '../datasets/%s.pickle' % expName
with open(dir, "rb") as fp:
    wholeset = pickle.load(fp)


rep  = wholeset[0]

chnNames = ['PZ', 'PO5', 'POZ', 'PO4', 'PO6', 'O1', 'OZ','O2']
chnINX = [wholeset[0]['channel'].index(i) for i in chnNames]

X = np.concatenate([sub['wn']['X'][:, chnINX] for sub in wholeset])
y = np.concatenate([sub['wn']['y'] for sub in wholeset])
S = rep['wn']['STI']
_class = np.unique(y)

In [18]:
from modeling import Code2EEG
srate = 240
winLEN = 1 
classNUM = 160
tmin, tmax = 0, .8
n_band = 1
# penalty
p = 0.9

code2EEG = Code2EEG(srate=srate, winLEN=winLEN, tmin=tmin, tmax=tmax, S=(S, _class), estimator=p, padding=True, n_band=n_band, component=1)
code2EEG.fit(X, y)

In [23]:
pattern = np.squeeze(code2EEG.enhanced)
STI = S

data=dict()
data['input'] = pattern
data['output'] = STI

savemat("../mat/FRF.mat", data)

(160, 240)

In [32]:
for sub in wholeset:
    name=sub['name']
    chnINX = [sub['channel'].index(i) for i in chnNames]
    data = dict()
    for paradigm in ['ssvep','wn','high']:
        X = sub[paradigm]['X'][:,chnINX,:]
        y = sub[paradigm]['y']

        X = np.stack([X[y==i] for i in np.unique(y)])
        X = np.transpose(X,axes=(0,2,3,1))

        data[paradigm] = X

    savemat("../mat/S_%s.mat"%name, data)
